In [3]:
import tensorflow as tf

from keras.applications.vgg16 import VGG16
import numpy as np
from keras import models
from keras import layers
# from keras.preprocessing.image import ImageDataGenerator      Deprecated
from keras import optimizers

ModuleNotFoundError: No module named 'tensorflow'

In [18]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [19]:
#x_train = x_train.reshape(x_train.shape[0], 28, 28)
#x_test = x_test.reshape(x_test.shape[0], 28, 28)

In [20]:
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)


In [21]:
def resize_images(images, labels):
    images = tf.image.resize(images, (150, 150))
    return images, labels

In [22]:
# Створення наборів даних
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(resize_images).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(resize_images).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

In [23]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [25]:
vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
vgg16.trainable = False

In [27]:
model = models.Sequential([
   vgg16,
   layers.Flatten(),
   layers.Dense(256, activation="relu"),
   layers.Dense(10, activation="softmax"),
])

In [28]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

In [30]:
model.fit(train_dataset,
          epochs=10, 
          batch_size=32, 
          validation_data=test_dataset)

Epoch 1/10


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Точність на тестовому наборі:', test_acc)

In [ ]:
model.save(
"/models/HW10/vgg16_basesd_model.hdf5")